In [ ]:
# default_exp video

In [ ]:
#export
import numpy as np
import cv2
from fastai.vision import *
import matplotlib.pyplot as plt

In [ ]:
# export
def track_frame(
    estimator,
    frame,
    box_half_size=25,
    box_scanning_step=5,
    ):
    """Tracks a frame box by box.
    
    Inputs:    
    network: the pretrained network
    frame: the frame to by analyzed
    box_half_size: half the size of the scanning box
    box_scanning_step: the size of the scanning step 
    
    Output:
    prediction_wrt_box: x, y and r coordiantes with respect to each box (pixels)
    prediction_wrt_frame: x, y and r coordinates with respect to each frame (pixels)
    boxes: the part of the frame corresponding to each box
    """  

    import numpy as np
    import cv2
    
    frame_width = frame.shape[1]
    frame_height = frame.shape[0]
    
    
    box_center_x = np.arange(box_half_size, 
                             frame_height - box_half_size, 
                             box_scanning_step)
    box_center_y = np.arange(box_half_size, 
                             frame_width - box_half_size, 
                             box_scanning_step)


    boxes = np.zeros((len(box_center_x), 
                      len(box_center_y),
                      box_half_size * 2 + 1, 
                      box_half_size * 2 + 1))  
    
    print(boxes.shape)
    prediction_wrt_box = np.zeros((len(box_center_x), 
                                   len(box_center_y), 
                                   3)) 
    prediction_wrt_frame = np.zeros((len(box_center_x), 
                                     len(box_center_y), 
                                     3)) 
    
    
    # Scanning over the frame row- and column-wise
    for j in range(len(box_center_x)):
        for k in range(len(box_center_y)):
            
            
            
            # Define the scanning box
            boxes[j, k] = frame[int(box_center_x[j] - box_half_size):int(box_center_x[j] + box_half_size + 1), 
                                int(box_center_y[k] - box_half_size):int(box_center_y[k] + box_half_size + 1)]
            
            box_predict = boxes[j, k]
            
            box_predict = cv2.resize(boxes[j, k], (51, 51))
            
            # Predict position of particle with respect to the scanning box
            prediction_wrt_box[j, k] = estimator(box_predict)[1] #network.predict(np.reshape(box_predict, (1, 51, 51, 1)))

            prediction_wrt_box[j, k][0] = prediction_wrt_box[j, k][0] * box_half_size + box_half_size
            prediction_wrt_box[j, k][1] = prediction_wrt_box[j, k][1] * box_half_size + box_half_size
            prediction_wrt_box[j, k][2] = prediction_wrt_box[j, k][2] * box_half_size

            prediction_wrt_frame[j, k][0] = prediction_wrt_box[j, k][0] + box_scanning_step * j
            prediction_wrt_frame[j, k][1] = prediction_wrt_box[j, k][1] + box_scanning_step * k
            prediction_wrt_frame[j, k][2] = prediction_wrt_box[j, k][2]

    return (prediction_wrt_box, prediction_wrt_frame, boxes)


In [ ]:
# hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_GenerateSyntheticImages.ipynb.
Converted 01_Models.ipynb.
Converted 02_video.ipynb.
Converted 03_measures.ipynb.
Converted 98_Display.ipynb.
Converted 99_cli.ipynb.
Converted E1_Track1.ipynb.
Converted E2_Track1fromN.ipynb.
Converted E3_multipleparticles.ipynb.
Converted index.ipynb.
Converted tutorial_CNN.ipynb.


In [ ]:
# export
def track(
    video_file_name,
    estimator,
    number_frames_to_be_tracked=1,
    box_half_size=25,
    box_scanning_step=5,
    frame_normalize=0,
    frame_enhance=1):
    """Track multiple particles in a video.
    
    Inputs:    
    video_file_name: video file
    network: the pre-trained network
    number_frames_to_be_tracked: number of frames to by analyzed from video begining. If number_frames is equal to 0 then the whole video is tracked.
    box_half_size: half the size of the scanning box. If box_half_size is equal to 0 then a single particle is tracked in a frame.
    box_scanning_step: the size of the scanning step
    frame_normalize: option to normalize the frame before tracking.
    frame_enhance: option to enhance the frame before tracking.
    
    Output:
    frames: frames from video
    predicted_positions_wrt_frame: x, y and r coordinates with respect to the all frames (pixels) 
    predicted_positions_wrt_box: x, y and r coordinates with respect to the boxes for all frames (pixels) 
    boxes_all: the part of the frame corresponding to each box for all frames
    number_frames_to_be_tracked: the number of frames that have been tracked. 
    """
    
    import cv2
    import numpy as np

    # Read the video file and its properties
    video = cv2.VideoCapture(video_file_name)

    if number_frames_to_be_tracked == 0:
        number_frames_to_be_tracked = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

    video_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    video_width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))

    # Initialize variables
    frames = np.zeros((number_frames_to_be_tracked, video_height, video_width))

   
    box_center_x = np.arange(box_half_size, 
                             video_height - box_half_size, 
                             box_scanning_step)
    box_center_y = np.arange(box_half_size, 
                             video_width - box_half_size, 
                             box_scanning_step)


    boxes_all = np.zeros((number_frames_to_be_tracked, 
                          len(box_center_x),
                          len(box_center_y), 
                          box_half_size * 2 + 1,
                          box_half_size * 2 + 1))

    predicted_positions_wrt_box = np.zeros((number_frames_to_be_tracked,
                                           len(box_center_x), 
                                           len(box_center_y), 
                                           3))
    predicted_positions_wrt_frame = np.zeros((number_frames_to_be_tracked,
                                             len(box_center_x), 
                                             len(box_center_y), 
                                             3))

    # Track the positions of the particles frame by frame
    for i in range(number_frames_to_be_tracked):

        # Read the current frame from the video
        (ret, frame) = video.read()
        
        # Normalize the frame
        if frame_normalize == 1:
            frame = cv2.normalize(frame, None, 0, 255, cv2.NORM_MINMAX)

        # Convert color image to grayscale.
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) / 255
        
        frame = frame * frame_enhance

        # Generate the scanning boxes and predict particle position in each box

        (prediction_wrt_box, prediction_wrt_frame, boxes) = track_frame(estimator, frame, box_half_size, box_scanning_step)


        frames[i] = frame
            
        predicted_positions_wrt_box[i] = prediction_wrt_box
        predicted_positions_wrt_frame[i] = prediction_wrt_frame
        boxes_all[i] = boxes

    # Release the video
    video.release()
    

    return (number_frames_to_be_tracked, frames, predicted_positions_wrt_frame, predicted_positions_wrt_box, boxes_all)



In [ ]:
# export
def plot_tracked_scanning_boxes(
    frame_to_be_shown,
    rows_to_be_shown,
    columns_to_be_shown,
    boxes_all,
    predicted_positions_wrt_box,
    box_half_size=25,
    ): 
    """Plot tracked scanning boxes over a range of frames.
    
    Inputs:    
    frame_to_be_shown: the range of frames to be shown
    rows_to_be_shown: the range of rows to be shown 
    columns_to_be_shown: the range of columns to be shown 
    boxes_all: the part of the frame corresponding to each box for all frames
    predicted_positions_wrt_box: x, y and r coordinates with respect to boxes for all frames (pixels)
    box_half_size: half the size of the scanning box
    
    Output: none
    """

    import matplotlib.pyplot as plt
       
    plt.figure(10)

    for i in list(frame_to_be_shown):
        for j in list(rows_to_be_shown):
            for k in list(columns_to_be_shown):

                plt.subplot(1, 2, 1)
                plt.imshow(boxes_all[i, j, k],
                           cmap='gray', 
                           vmin=0, 
                           vmax=1)
                
                plt.plot(predicted_positions_wrt_box[i, j, k, 1],
                         predicted_positions_wrt_box[i, j, k, 0], 
                         'ob')
                plt.xlabel('y (px)', fontsize=16)
                plt.ylabel('x (px)', fontsize=16)
                
                
                plt.subplot(1, 2, 2)

                plt.text(0, .8, 'frame = %1.0f' % i, fontsize=16)
                plt.text(0, .7, 'row = %1.0f' % j, fontsize=16)
                plt.text(0, .6, 'column = %1.0f' % k, fontsize=16)
                
                plt.text(0, .4, 'particle center x = %5.2f px' % predicted_positions_wrt_box[i, j, k, 0], 
                         fontsize=16, color='b')
                plt.text(0, .3, 'particle center y = %5.2f px' % predicted_positions_wrt_box[i, j, k, 1], 
                         fontsize=16, color='b')
                plt.text(0, .2, 'particle radius = %5.2f px' % predicted_positions_wrt_box[i, j, k, 2], 
                         fontsize=16, color='b')
                plt.axis('off')

                
                plt.show()




In [ ]:
np.ones()

In [ ]:
# export
def centroids(
    particle_positions_x,
    particle_positions_y,
    particle_radial_distance,
    particle_interdistance,
    ):
    """Calculate centroid of the particles by taking the mean x and y positions.
    
    Inputs:    
    x_particle_positions: the predicted x-positions for the particles (many for each particle)
    y_particle_positions: the predicted y-positions for the particles (many for each particle)
    particle_radial_distance: the radial distance of the particle from the center of the scanning box
    particle_max_interdistance: the maximum distance between predicted points for them to belong to the same particle
    
    Output: 
    x_centroid: the x coordinate of the centroid for each particle 
    y_centroid: the y coordinate of the centroid for each particle
    """

    import numpy as np

    def ppos_same_particle(p1,p2):
        return ((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2) < rmin
            
    particle_number = 0
    particle_index = 0
    particle_numbers = -np.ones(len(particle_positions_x)) #[-1,-1,-1,-1]
    
    # Sort all predicted points to correct particle 
    
    # number the particles
    while particle_numbers[np.argmin(particle_numbers)] == -1:
        particle_index = np.argmin(particle_numbers) # index of lowest particle num
        particle_numbers[particle_index] = particle_number # value of lowest particle num
        particle_number += 1 # effectively 'select' this particle

        for j in range(len(particle_positions_x)):

            if ppos_same_particle(ppos[j], ppos[pi], particle_rmin):
                particle_numbers[j] = particle_numbers[particle_index]
                
#             if (particle_positions_x[j] - particle_positions_x[particle_index]) ** 2 \
#                 + (particle_positions_y[j] - particle_positions_y[particle_index]) ** 2 \
#                 < particle_interdistance ** 2:
#                 particle_numbers[j] = particle_numbers[particle_index]

    centroid_x = np.zeros(int(np.amax(particle_numbers)) + 1)
    centroid_y = np.zeros(int(np.amax(particle_numbers)) + 1)

    particle_number = 0
    while max(particle_numbers) >= particle_number:
        points_x = particle_positions_x[np.where(particle_numbers == particle_number)]
        points_y = particle_positions_y[np.where(particle_numbers == particle_number)]
        distance_from_center = particle_radial_distance[np.where(particle_numbers == particle_number)]

        # Calculate centroids
        _len = len(points_x)
        centroid_x[particle_number] = sum(points_x) / _len
        centroid_y[particle_number] = sum(points_y) / _len

        particle_number += 1

    return (centroid_x, centroid_y)


In [ ]:
# export
def show_tracked_frames(
    particle_radial_distance_threshold,
    particle_maximum_interdistance,
    number_frames_to_be_shown,
    frames,
    predicted_positions_wrt_frame,
    ):
    """Show the frames with the predicted positions and centroid positions.
    
    Inputs:    
    particle_radial_distance: the radial distance of the particle from the center of the scanning box
    particle_maximuminterdistance: the maximum distance between predicted points for them to belong to the same particle
    number_frames_to_be_shown: number of frames to be shown
    frames: frames from video
    predicted_positions_wrt_frame: x, y and r coordinates with respect to frames (pixels) 
    
    Output: none
    """
    
    import numpy as np
    import matplotlib.pyplot as plt

    particle_positions = []
    particle_centroids = []
    particle_radial_distance = []

    for i in range(number_frames_to_be_shown):

        particle_positions_x = []
        particle_positions_y = []

        # Show frame
        plt.figure(figsize=(10, 10))
        plt.imshow(frames[i], cmap='gray', vmin=0, vmax=1)

        # Threshold the radial distance of the predicted points
        for j in range(0, predicted_positions_wrt_frame.shape[1]):
            for k in range(0, predicted_positions_wrt_frame.shape[2]):

                if predicted_positions_wrt_frame[i, j, k, 2] \
                    < particle_radial_distance_threshold:

                    # Plot the predicted points
                    plt.plot(predicted_positions_wrt_frame[i, j, k, 1],
                             predicted_positions_wrt_frame[i, j, k, 0], '.b')
                    particle_positions_x = \
                        np.append(particle_positions_x,
                                  predicted_positions_wrt_frame[i, j, k, 0])
                    particle_positions_y = \
                        np.append(particle_positions_y,
                                  predicted_positions_wrt_frame[i, j, k, 1])
                    particle_radial_distance = \
                        np.append(particle_radial_distance,
                                  predicted_positions_wrt_frame[i, j, k, 2])

        particle_positions.append([])
        particle_positions[i].append(particle_positions_x)
        particle_positions[i].append(particle_positions_y)

        # Calculate the centroid positions
        (centroids_x, centroids_y) = centroids(particle_positions_x,
                                               particle_positions_y, 
                                               particle_radial_distance,
                                               particle_maximum_interdistance)

        particle_centroids.append([])
        particle_centroids[i].append(centroids_x)
        particle_centroids[i].append(centroids_y)

        # Plot the centroid positions
        plt.plot(
            centroids_y,
            centroids_x,
            'o',
            color='#e6661a',
            fillstyle='none',
            markersize=10,
            )
        
    return #(particle_positions, particle_centroids)


In [ ]:
# export
def track_single_particle(
    video_file_name,
    estimators,
    number_frames_to_be_tracked=0,
    frame_normalize=0,
    frame_enhance=1,
    use_cv2=False):
    """Track single particlee in a video.
    
    Inputs:    
    video_file_name: video file
    network: the pre-trained network
    number_frames_to_be_tracked: number of frames to by analyzed from video begining. If number_frames is equal to 0 then the whole video is tracked.
        
    Output:
    frames: frames from video
    predicted_positions: x, y and r coordinates with respect to the all frames (pixels) 
    """
    
    from deeptracktorch.simg import npimg2tensor



    if use_cv2:
        
        # Read the video file and its properties
        video = cv2.VideoCapture(video_file_name)
        fps = video.get(cv2.CAP_PROP_FPS)
        print(fps)

        if number_frames_to_be_tracked == 0:
            number_frames_to_be_tracked = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

        video_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
        video_width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))  

        # Track the positions of the particles frame by frame
        for i in range(number_frames_to_be_tracked):

            # Read the current frame from the video
            (ret, frame) = video.read()

            # Normalize the frame
            if frame_normalize == 1:
                frame = cv2.normalize(frame, None, 0, 255, cv2.NORM_MINMAX)

            # Convert color image to grayscale.
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) / 255

            # Enhance the frame
            frame = frame * frame_enhance

            frame = cv2.resize(frame,(51,51))

            scale_x = video_width / 51
            scale_y = video_height / 51

            estimated_positions = {k:np.empty((number_frames_to_be_tracked,3)) for (k,v) in estimators.items()}

            # apply methods to estimate particle positions
            image = np.array(frame)

            for e in estimators: estimated_positions[e][i,:]=estimators[e](frame)
    
    else:
        import av
        container = av.open(video_file_name)

        if number_frames_to_be_tracked == 0:
            number_frames_to_be_tracked = container.streams.video[0].frames
        
        estimated_positions = {k:np.empty((number_frames_to_be_tracked,3)) for (k,v) in estimators.items()}

        for i, frame in enumerate(container.decode(video=0)):
            image = frame.to_image().resize((51,51)).convert('L')
            image = np.array(image) / 255
    
            for e in estimators: estimated_positions[e][i,:]=estimators[e](image)
    
    return estimated_positions



In [ ]:
### Define the video file to be tracked
video_file_name = '../../DeepTrack 1.0/DeepTrack - Example 2 - Optically Trapped Particle Good.mp4'

### Define the number of frames to be tracked
number_frames_to_be_tracked = 2

### Preprocess the images
frame_normalize = 0
frame_enhance = 1

### Track the video
(number_tracked_frames, frames, predicted_positions) = track_single_particle(
    video_file_name, 
    learner, 
    number_frames_to_be_tracked,
    frame_normalize,
    frame_enhance)

print(predicted_positions)

NameError: name 'learner' is not defined

In [ ]:
# export
def show_tracked_frames_single_particle(
    number_frames_to_be_shown,
    frames,
    predicted_positions,
    ):
    """Show the frames with the predicted position.
    
    Inputs:    
    number_frames_to_be_shown: number of frames to be shown
    frames: frames from video
    predicted_positions: x, y and r coordinates (pixels) 
    
    Output: none
    """
    
    import numpy as np
    import matplotlib.pyplot as plt


    for i in range(number_frames_to_be_shown):

        # Show frame
        plt.figure(figsize=(10, 10))
        plt.imshow(frames[i], cmap='gray', vmin=0, vmax=1)


        # Plot the predicted points
        plt.plot(predicted_positions[i, 1],
                 predicted_positions[i, 0], 
                 '.',
                 color='#e6661a',
                 #fillstyle='none',
                 markersize=20,
                 )

In [ ]:
# export
def particle_positions(
    particle_number=2,
    first_particle_range=0.5,
    other_particle_range=1,
    particle_distance=50,
    ):
    """Generates multiple particle x- and y-coordinates with respect to each other.
    
    Inputs:  
    particle_number: number of particles to generate coordinates for
    first_particle_range: allowed x- and y-range of the centermost particle
    other_particle_range: allowed x- and y-range for all other particles
    particle_distance: particle interdistance
    
    Output:
    particles_center_x: list of x-coordinates for the particles
    particles_center_y: list of y-coordinates for the particles
    """

    from numpy.random import normal
    from numpy.random import uniform
    from numpy import insert
    from numpy import array
    from itertools import combinations
    from numpy import linalg
    
    ### Centermost particle
    target_particle_center_x = uniform(-first_particle_range,
            first_particle_range)
    target_particle_center_y = uniform(-first_particle_range,
            first_particle_range)

    target_center_distance = (target_particle_center_x ** 2
                              + target_particle_center_y ** 2) ** 0.5

    ### Other particles
    while True:
        particles_center_x = uniform(-other_particle_range,
                other_particle_range, particle_number - 1)  
        particles_center_y = uniform(-other_particle_range,
                other_particle_range, particle_number - 1) 

        center_distance = (particles_center_x ** 2 + particles_center_y
                           ** 2) ** 0.5

        ### Force all other particles to be further away from the center than the centermost particle
        if any(t < target_center_distance for t in center_distance):
            continue

        particles_center_x = insert(particles_center_x, 0,
                                    target_particle_center_x)
        particles_center_y = insert(particles_center_y, 0,
                                    target_particle_center_y)

        particle_centers = array([particles_center_x,
                                 particles_center_y])

        ### Force all particles to be a certain distance from each other
        if all(linalg.norm(p - q) > particle_distance for (p, q) in
               combinations(particle_centers, 2)):
            break

    return (particles_center_x, particles_center_y)

